In [1]:
import pandas as pd
import numpy as np

In [8]:
df1 = pd.read_csv(r'../crsp/sp500_daily.csv')
df1.head()

C:\Users\Patrick\AppData\Local\Temp\ipykernel_26372\756145761.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv(r'../crsp/sp500_daily.csv')


,Unnamed: 0,date,permno,comnam,ncusip,shrcd,exchcd,hsiccd,ticker,gvkey,iid,start,ending,ret,retx,prc,shrout,vol,cfacpr,cfacshr
0,1,2000-01-03,11703,DU PONT E I DE NEMOURS & CO,26353410,11,1,2879,DD,4087,1,1957-03-01,2017-08-31,-0.013283,-0.013283,65.0000,1043131.0,2791500.0,1.053752,1.0
1,2,2000-01-03,24766,NORTHROP GRUMMAN CORP,66680710,11,1,3812,NOC,7985,1,1985-06-13,2024-12-31,-0.021965,-0.021965,52.8750,69489.0,152100.0,2.220592,2.0
2,4,2000-01-03,24563,TEXAS UTILITIES CO,88284810,11,1,4911,TXU,10507,1,1957-03-01,2007-10-09,-0.028120,-0.028120,34.5625,276407.0,1029700.0,2.000000,2.0
3,5,2000-01-03,40416,AVON PRODUCTS INC,05430310,11,1,2844,AVP,1920,1,1967-05-18,2015-03-20,-0.037879,-0.037879,31.7500,242304.0,944400.0,2.000000,2.0
4,6,2000-01-03,26710,LOEWS CORP,54042410,11,1,6331,LTR,6781,1,1995-05-10,2024-12-31,-0.026777,-0.026777,59.0625,106050.0,413700.0,6.000000,6.0


In [11]:
# Parse dates and sort
df1['date'] = pd.to_datetime(df1['date'])
df1 = df1.sort_values(['date', 'permno'])

# Optional: keep only common shares on main exchanges (standard CRSP filters)
if 'shrcd' in df1.columns:
    df1 = df1[df1['shrcd'].isin([10, 11])]
if 'exchcd' in df1.columns:
    df1 = df1[df1['exchcd'].isin([1, 2, 3])]

# Drop missing returns
df1 = df1.dropna(subset=['ret'])

print("Unique dates:", df1['date'].nunique())
print("Unique permnos:", df1['permno'].nunique())

# Pick a stable small universe: permnos with the longest history
counts = df1.groupby('permno')['date'].nunique().sort_values(ascending=False)
K_UNIVERSE = 30   # you can tweak this (20–50 is fine for Week 3)
universe_permnos = counts.head(K_UNIVERSE).index.tolist()

df_sub = df1[df1['permno'].isin(universe_permnos)].copy()

# Date x permno matrix of returns
ret_panel = df_sub.pivot(index='date', columns='permno', values='ret').sort_index()
""
# Drop any dates with missing returns in this universe (simplest approach)
ret_panel = ret_panel.dropna(how='any')

print("Panel shape (T x N):", ret_panel.shape)
ret_panel.head()


Unique dates: 6289
Unique permnos: 978
Panel shape (T x N): (6289, 30)


permno,10104,10107,10145,10516,11308,13856,13901,48506,48653,48725,...,58683,59176,59248,59328,59408,59459,60206,60506,61399,92655
date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,0.054099,-0.001606,-0.017335,-0.010309,-0.032189,0.046099,0.019022,-0.023305,-0.076336,-0.014306,...,-0.015504,-0.054135,-0.034524,0.056948,-0.034869,-0.020408,0.011765,-0.018336,-0.066946,0.011765
2000-01-04,-0.088360,-0.033780,-0.017641,-0.010417,-0.008869,-0.025424,0.008000,-0.032538,0.008264,0.000000,...,-0.027559,-0.037758,-0.012331,-0.046695,-0.059355,-0.013258,-0.011628,-0.045977,-0.053812,-0.012791
2000-01-05,-0.052815,0.010544,-0.013468,-0.015790,0.019016,-0.024348,-0.005291,-0.017937,0.024590,-0.001451,...,-0.004049,-0.025361,0.047441,0.008289,0.010974,-0.007678,-0.037647,0.040663,-0.001185,-0.002356
2000-01-06,-0.058824,-0.033498,0.019340,0.005348,0.001098,0.044563,-0.018617,0.004566,0.040000,0.061047,...,-0.004065,0.020374,-0.011919,-0.058296,0.071913,0.019342,0.002445,-0.013025,-0.002372,0.036600
2000-01-07,0.076823,0.013068,0.053571,0.015957,0.065790,0.044369,0.027100,0.022727,0.076923,-0.006849,...,0.036735,0.014559,0.021713,0.041270,-0.013924,0.039848,0.024390,0.038123,0.022592,0.117312
